<a href="https://colab.research.google.com/github/SreeniEadara/embedded-hophacks-22/blob/main/HopHacks_9_17_22_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
############
# IMPORTS
############
import RPi.GPIO as GPIO
from RpiMotorLib import RpiMotorLib
import time
#import numpy as np




############
# Motor Setup
############
dir_pin = 13
step_pin = 19
Enable_pin = 26

stepMotor = RpiMotorLib.A4988Nema(dir_pin, step_pin, (21,21,21), "DRV8825") ####################################################################################
GPIO.setup(Enable_pin,GPIO.OUT) # set enable pin as output ####################################################################################


#dir_Bools = [False,True]
#dir_Bools[ii%2] #Can do 2 cup dispensing

GPIO.output(Enable_pin,GPIO.LOW) # pull enable to low to enable motor


############
# Server Motor Control
############








from flask import Flask
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast
import time

name = "main"

app = Flask(name)
api = Api(app)

class Medications(Resource):
    def get(self):
        data = pd.read_csv('medications.csv')  # read local CSV
        data = data.to_dict()  # convert dataframe to dict
        return {'data': data}, 200  # return data and 200 OK

    def post(self):
        parser = reqparse.RequestParser()  # initialize
        parser.add_argument('name', required=True)  # add args
        parser.add_argument('frequency', required=True)
        parser.add_argument('max_quantity', required=True)
        parser.add_argument('unit_quantity', required=True)
        args = parser.parse_args()  # parse arguments to dictionary

        # read our CSV
        data = pd.read_csv('medications.csv')

        # create new dataframe containing new values
        new_data = pd.DataFrame({
            'name': [args['name']],
            'frequency': [args['frequency']],
            'max_quantity': [args['max_quantity']],
            'unit_quantity': [args['unit_quantity']]
        })


        MedDispensingTime=[]
        maxDoseNum = new_data['max_quantity']/new_data['unit_quantity']
        for i in range(maxDoseNum):
          MedDispensingTimeTemp = 8.00 + (new_data['frequency']*i)
          MedDispensingTimeTemp =  MedDispensingTimeTemp % 24
          MedDispensingTime.append(MedDispensingTimeTemp)
        InvertedMedDispensingTime = MedDispensingTime.reverse()
        dosesLeft=maxDoseNum

        new_data.insert(2, "dosesLeft", dosesLeft, True)
        new_data.insert(2, "InvertedMedDispensingTime", InvertedMedDispensingTime, True)


        # add the newly provided values
        data = data.append(new_data, ignore_index=True)
        data.to_csv('medications.csv', index=False)  # save back to CSV
        '''
        sendString = "{
          \"rspCde\" : 0,
          \"rspMsg\": \"Success\",
          \"medications\": [
            "
        rows = len(data)
        columns = len(data.columns)

        for i + 1 in range(rows)
          for j in range(columns)

        '''
        return {'data': data.to_dict()}, 200  # return data with 200 OK

        for i in range(10):
          stepMotor.motor_go(False,"Full",200, .0005, False,.05)

        '''
        while True:
          for index, row in data.iterrows():
            locTime = time.localtime()
            TimeFloat = (locTime[3] + (locTime[4]/60) + (locTime[5]/3600))
            if dosesLeft > 0:
              MedDispensingTimeCurrent = row['InvertedMedDispensingTime'][-1]
              if locTime >= MedDispensingTimeCurrent and locTime <= MedDispensingTimeCurrent + 0.01 :
                for i in range(int(unit_quantity)):
                  stepMotor.motor_go(False,"Full",200, .0005, False,.05)
                  dosesLeft = dosesLeft - 1
                  InvertedMedDispensingTime = row['InvertedMedDispensingTime'].pop()
                  new_data.insert(2, "dosesLeft", dosesLeft, True)
                  new_data.insert(2, "InvertedMedDispensingTime", InvertedMedDispensingTime, True)
                time.sleep(10)
            else:
              GPIO.cleanup()
          '''

          GPIO.cleanup()

api.add_resource(Medications, '/medications')  # add endpoints

if name == 'main':
    app.run(host='0.0.0.0')  # run our Flask app




